# Part Four
## Building a web farm for development and testing

### Committing previous changes

In [1]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
centos6             serverv1            be824f20375e        21 minutes ago      373 MB
centos6             baseweb             34079efab3e4        2 hours ago         373 MB
centos              centos6             8315978ceaaa        4 months ago        195 MB


In [2]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                           PORTS                  NAMES
af3ac3489de7        centos6:serverv1    "/bin/bash"         14 minutes ago      Up 14 minutes                    0.0.0.0:8081->80/tcp   externalweb
62043409c68d        centos6:serverv1    "/bin/bash"         19 minutes ago      Up 18 minutes                    80/tcp                 contserv1
f9fd0dbebbeb        centos6:baseweb     "/bin/bash"         About an hour ago   Exited (255) 32 minutes ago      0.0.0.0:8080->80/tcp   webtest
695d1ddda23e        centos6:baseweb     "/bin/bash"         2 hours ago         Exited (137) About an hour ago                          fervent_wright
548f70d44cb2        centos:centos6      "/bin/bash"         3 hours ago         Exited (137) 2 hours ago                                reverent_pasteur


**We can see our container externalweb, and we want to commit it to save it**

In [3]:
docker commit externalweb centos6:finalwebv1

sha256:758e7bcba59c48e0b2cbb4563881cef79755bca73ab41d723967ff177c922e2d


**This is the base container which we use to base all the systems from**

In [4]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
centos6             finalwebv1          758e7bcba59c        22 seconds ago      373 MB
centos6             serverv1            be824f20375e        22 minutes ago      373 MB
centos6             baseweb             34079efab3e4        2 hours ago         373 MB
centos              centos6             8315978ceaaa        4 months ago        195 MB


**We can see our image is there - so we're good :)**

## Managing Web Directory
We would normally not have the git repo with everything there.
We'll move our dockerwww to dockergit, which will be our github repository, so that it's separate from our build repository.

In [5]:
ls

Docker Notebook - Part 1 - Creating and removing containers + images.ipynb
Docker Notebook - Part 2 - Setting up the website.ipynb
Docker Notebook - Part 3 - Configuring access internally + externally.ipynb
Docker Notebook - Part 4 - Building a web farm for development and testing.ipynb
README.md
bluefreedom3.zip
dockerwww
img
requirements.txt
venv


In [6]:
mv dockerwww dockergit

In [12]:
# We want to clone the repository to treat this as if it was the prod version
rm -rf dockerwwwfull
git clone $(pwd) dockerwwwfull
mv dockerwwwfull/dockergit dockerwww
rm -rf dockerwwwfull

Cloning into 'dockerwwwfull'...
done.


## Starting docker
Now let's start our docker instances

In [19]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
centos6             finalwebv1          758e7bcba59c        6 minutes ago       373 MB
centos6             serverv1            be824f20375e        28 minutes ago      373 MB
centos6             baseweb             34079efab3e4        2 hours ago         373 MB
centos              centos6             8315978ceaaa        4 months ago        195 MB


## **Launch PROD instance 2**

In [26]:
docker run -dit --name=devweb1 -p 8081:80 -v $(pwd)/dockerwww:/var/www/html centos6:finalwebv1 /bin/bash


f0396e16605a5b5e6abaefcc6640db58f8420591061986172d401ad2ddcb617b


In [27]:
docker ps

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS              PORTS                  NAMES
f0396e16605a        centos6:finalwebv1   "/bin/bash"         5 seconds ago       Up 4 seconds        0.0.0.0:8081->80/tcp   devweb1


## Launch PROD instance 1

In [28]:
docker run -dit --name=devweb2 -p 8082:80 -v $(pwd)/dockerwww:/var/www/html centos6:finalwebv1 /bin/bash


1c8098f3ec121a0679487e38459cc26b96178b71e96bb3741567e2ab9871111a


In [29]:
docker ps

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS              PORTS                  NAMES
1c8098f3ec12        centos6:finalwebv1   "/bin/bash"         4 seconds ago       Up 3 seconds        0.0.0.0:8082->80/tcp   devweb2
f0396e16605a        centos6:finalwebv1   "/bin/bash"         43 seconds ago      Up 43 seconds       0.0.0.0:8081->80/tcp   devweb1


**Nowboth are running!**

## Inspect for IP

In [32]:
docker inspect devweb1 | grep IPAddress

            "SecondaryIPAddresses": null,
            "IPAddress": "172.17.0.2",
                    "IPAddress": "172.17.0.2",


In [33]:
docker inspect devweb2 | grep IPAddress

            "SecondaryIPAddresses": null,
            "IPAddress": "172.17.0.3",
                    "IPAddress": "172.17.0.3",


**We should be able to visit both of the IP addresses, as well as through both of the port of the http://localhost:8081 and http://localhost:8082**

## Proxy and load balance
We will install nginx to proxy and load balance the connection to these both containers. 
We will proxy port 80 into both port 8081 and 8082
### Make sure you have NGINX installed!
We should now have:
* NGINX running in port 8080
* One server running in port 8081
* Another server running in port 8082

All we have to do is to access the NGINX file in sites-available, and add this to our file:

``` bash

upstream containerapp {
    server 127.0.0.1:8081;
    server 127.0.0.1:8082;
}

server {
    listen *:80;

    server_name 127.0.0.1;
    index index.html index.htm index.php;

    access_log /Users/alejandrosaucedo/Programming/devnull/helloworlddocker/localweb.log;
    error_log /Users/alejandrosaucedo/Programming/devnull/helloworlddocker/localerr.log;

    location / {
        proxy_pass http://containerapp;
    }
}

```

### Multi-farm web setup!
This has allowed us to manage a multi-container setup that provides load balacing to our two apps from external access!
